In [57]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/booking-com-hotel-reviews/booking_reviews copy.csv


In [58]:
!pip install nltk
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import statsmodels.formula.api as smf

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [59]:
data = pd.read_csv('/kaggle/input/booking-com-hotel-reviews/booking_reviews copy.csv')

In [60]:
print(data.isnull().sum())

index                  0
review_title           1
reviewed_at          105
reviewed_by          105
images             25737
crawled_at           289
url                  289
hotel_name           289
hotel_url            289
avg_rating           289
nationality          305
rating               289
review_text          289
raw_review_text      473
tags                 473
meta                 473
dtype: int64


In [61]:
data['images'] = data['images'].notna()
data = data.dropna(subset=['review_title'])
data = data.drop(columns=['url', 'crawled_at', 'hotel_url'])
data = data.dropna(subset=['hotel_name'])
print(data.isnull().sum())

index                0
review_title         0
reviewed_at          0
reviewed_by          0
images               0
hotel_name           0
avg_rating           0
nationality         16
rating               0
review_text          0
raw_review_text    184
tags               184
meta               184
dtype: int64


In [62]:
analyzer = SentimentIntensityAnalyzer()

# Function to assign sentiment scores
def assign_sentiment(review_text):
    sentiment = analyzer.polarity_scores(review_text)
    if sentiment['compound'] > 0.2:
        return 3  # Positive
    elif sentiment['compound'] < -0.2:
        return 1  # Negative
    else:
        return 2  # Neutral


data['sentiment_score'] = data['review_text'].apply(assign_sentiment)

In [63]:
data.head()

,index,review_title,reviewed_at,reviewed_by,images,hotel_name,avg_rating,nationality,rating,review_text,raw_review_text,tags,meta,sentiment_score
0,0,Exceptional,11 July 2021,Kyrylo,False,Villa Pura Vida,9.7,Poland,10.0,"Everything was perfect! Quite, cozy place to r...","<p class=""review_pos ""><svg aria-label=""Positi...",Business trip~Solo traveller~Junior Suite~Stay...,"{'language': 'en-gb', 'source': 'https://www.b...",3
1,1,I highly recommend this b&b! We enjoyed it a lot!,24 November 2019,Dimitri,False,Villa Pura Vida,9.7,Belgium,9.0,Very friendly host and perfect breakfast!,"<p class=""review_pos ""><svg aria-label=""Positi...",Leisure trip~Couple~Deluxe Suite~Stayed 1 nigh...,"{'language': 'en-gb', 'source': 'https://www.b...",3
2,2,Exceptional,3 January 2020,Virginia,False,Hydro Palace Apartment,9.2,United Kingdom,10.0,It was just what we wanted for a week by the b...,"<p class=""review_neg ""><svg aria-label=""Negati...",Leisure trip~Couple~Apartment with Sea View~St...,"{'language': 'en-gb', 'source': 'https://www.b...",3
3,3,My stay in the house was a experiencing bliss ...,8 September 2019,Kannan,False,Villa Pura Vida,9.7,Netherlands,10.0,My stay in the house was a experiencing bliss ...,"<p class=""review_pos ""><svg aria-label=""Positi...",Business trip~Solo traveller~Junior Suite~Stay...,"{'language': 'en-gb', 'source': 'https://www.b...",3
4,4,One bedroom apartment with wonderful view and ...,23 June 2019,Sue,True,Hydro Palace Apartment,9.2,South Africa,9.2,The building itself has a very musty smell in ...,"<p class=""review_neg ""><svg aria-label=""Negati...",Leisure trip~People with friends~Apartment wit...,"{'language': 'en-gb', 'source': 'https://www.b...",3


In [64]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

# Preprocess the text
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    # Remove HTML tags
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    # Remove special characters and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stop words
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

data['processed_review_text'] = data['review_text'].apply(preprocess_text)

vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

# Fit and transform the data
dtm = vectorizer.fit_transform(data['processed_review_text'])

# Apply LDA
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(dtm)

# Get the top words for each topic
def get_top_words(model, feature_names, n_top_words):
    topics = []
    for topic_idx, topic in enumerate(model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        topics.append(" ".join(top_words))
    return topics

feature_names = vectorizer.get_feature_names_out()
top_words_per_topic = get_top_words(lda, feature_names, 10)

# Assign topics to reviews
topic_assignments = lda.transform(dtm)
data['topic'] = [np.argmax(row) for row in topic_assignments]

# Analyze sentiment for each topic
for topic_id in range(len(top_words_per_topic)):
    topic_reviews = data[data['topic'] == topic_id]['review_text']
    topic_sentiment = topic_reviews.apply(lambda x: analyzer.polarity_scores(x)['compound'])
    print(f"Topic {topic_id}: {top_words_per_topic[topic_id]}")
    print(f"Average Sentiment: {topic_sentiment.mean()}")


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Topic 0: breakfast room great stay nice hotel location friendly staff lovely
Average Sentiment: 0.7410356767904012
Topic 1: check apartment hotel time stay booking place kitchen day reception
Average Sentiment: 0.4252983808290156
Topic 2: available comments review kamer polished chap wery klein lokation smal
Average Sentiment: -0.29514606202153887
Topic 3: location good staff breakfast great clean hotel room friendly nice
Average Sentiment: 0.6716202533532042
Topic 4: room bed bathroom shower location good nice hotel staff clean
Average Sentiment: 0.3487114341846758


In [75]:
import statsmodels.formula.api as smf

# Select the relevant columns
data_analysis = data[['hotel_name', 'rating', 'sentiment_score']]

# Perform ANOVA
anova_table = smf.ols('rating ~ hotel_name + sentiment_score', data=data_analysis).fit()

# Analyze results
hotel_name_pvalues = anova_table.summary2().tables[1].iloc[1:, 4]
significant_effects = anova_table.pvalues < 0.0000001

significant_hotels = anova_table.params.index[significant_effects & (anova_table.params.index != 'Intercept')]
significant_hotels_clean = [hotel_name.replace("hotel_name[T.", "").replace("]", "") for hotel_name in significant_hotels if 'hotel_name[T.' in hotel_name]

print("Significant hotels:")
for hotel in significant_hotels_clean:
    print(f"- {hotel}")




Significant hotels:
- Budget Flats Leuven
- Budget Logies Grenadine
- CASA DI MARCO
- EXIT 29
- Flat Brussels center
- Hotel Albergo
- Hotel Class'Eco Namur
- Hotel Class'eco Liège
- Hotel Du Parlement
- Hotel Eurocap
- Hotel Evergreen
- Hotel La Potinière
- Hotel Le New Matinal
- Hotel Midi-Zuid
- Hotel Mirabeau
- Hotel Ramada Brussels Woluwe
- Roture 88
- Smartflats City - Royal
- Smartflats Design - Antwerp Central
- The Helmet Hotel


In [77]:
def preprocess_text(text):
    return re.sub(r'[^\w\s]', '', str(text).lower())

def analyze_reviews(data):
    if isinstance(data, dict):
        data = pd.DataFrame(data)
    
    if 'review_text' not in data.columns or 'hotel_name' not in data.columns:
        raise ValueError("The dataframe must contain 'review_text' and 'hotel_name' columns")
    
    data['processed_text'] = data['review_text'].apply(preprocess_text)
    
    aspects = {
        'Cleanliness': ['clean', 'dirty', 'dust', 'spotless', 'hygiene'],
        'Staff': ['staff', 'service', 'friendly', 'helpful', 'rude'],
        'Amenities': ['amenities', 'facilities', 'pool', 'gym', 'wifi'],
        'Price': ['price', 'expensive', 'cheap', 'value', 'worth']
    }
    
    positive_words = ['good', 'great', 'excellent', 'amazing', 'wonderful', 'fantastic']
    negative_words = ['bad', 'poor', 'terrible', 'awful', 'disappointing', 'horrible']
    
    results = {hotel: {aspect: {'positive': 0, 'negative': 0} for aspect in aspects} 
               for hotel in data['hotel_name'].unique()}
    
    for _, row in data.iterrows():
        hotel = row['hotel_name']
        words = row['processed_text'].split()
        
        for aspect, aspect_words in aspects.items():
            aspect_mentioned = any(word in words for word in aspect_words)
            if aspect_mentioned:
                pos_count = sum(word in words for word in positive_words)
                neg_count = sum(word in words for word in negative_words)
                
                if pos_count > neg_count:
                    results[hotel][aspect]['positive'] += 1
                elif neg_count > pos_count:
                    results[hotel][aspect]['negative'] += 1
    
    return results

def create_sentiment_csv(results, output_file):
    data = []
    for hotel, aspects in results.items():
        row = {'Hotel Name': hotel}
        for aspect, sentiment in aspects.items():
            if sentiment['positive'] > sentiment['negative']:
                row[aspect] = '+'
            elif sentiment['negative'] > sentiment['positive']:
                row[aspect] = '-'
            else:
                row[aspect] = ' '
        data.append(row)
    
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)
    print(f"CSV file '{output_file}' has been created.")

# Load your main data here
main_data = pd.read_csv('/kaggle/input/booking-com-hotel-reviews/booking_reviews copy.csv')  # Replace with your actual data file

# Analyze reviews
analysis_results = analyze_reviews(main_data)

# Create the sentiment CSV file
create_sentiment_csv(analysis_results, 'hotel_sentiment_analysis.csv')

CSV file 'hotel_sentiment_analysis.csv' has been created.


In [78]:
df = pd.read_csv('/kaggle/working/hotel_sentiment_analysis.csv')
df

,Hotel Name,Cleanliness,Staff,Amenities,Price
0,Villa Pura Vida,+,+,+,+
1,Hydro Palace Apartment,+,,,
2,Love Nest Versailles - Unique Louis XIV decora...,+,,+,
3,Gite Namaste,,,,+
4,Hotel Saint Sauveur by WP Hotels,+,+,+,+
...,...,...,...,...,...
815,Smartflats City - Royal,,,+,
816,Glamping op het Zeugekot,,,,
817,Hotel Bilderdijk,+,+,,
818,B&B Roosendaelhof,,,,
